# **STANDARD NOTEBOOK TO GET RESULTS**

## 1. Clean the Data

In [38]:
import pandas as pd 
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [55]:
naive_portfolio = df_cleaned.mul(1/len(df_cleaned.columns)).mean(axis=1)

# Calcul de la PnL cumulée
pnl_cumul = np.cumprod(1 + naive_portfolio) - 1

# Calcul du Sharpe Ratio
# Supposons un taux sans risque de 0%
risk_free_rate = 0.0
annualized_return = pnl_cumul[-1] * 252 / len(pnl_cumul)  # 252 jours de trading par an
annualized_volatility = naive_portfolio.std() * np.sqrt(252)
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
sharpe_ratio

0.11212421195119217

In [56]:
pnl_cumul

03/01/2000   -0.000025
04/01/2000   -0.000041
05/01/2000   -0.000030
06/01/2000   -0.000019
07/01/2000    0.000003
                ...   
24/12/2020    0.000536
28/12/2020    0.000530
29/12/2020    0.000515
30/12/2020    0.000523
31/12/2020    0.000529
Length: 5279, dtype: float64

## 4. Get SP500 data for comparison

In [62]:
import process

start_date = '2000-01-06'
end_date = '2020-12-30'
year = '2001-2020'
clustering_method = 'naive'

sp500_PnL = process.get_sp500_PnL(start_date, end_date)
print(len(sp500_PnL))
print(len(pnl_cumul))

[*********************100%%**********************]  1 of 1 completed
5279
5279


## 5. Plot Portfolio Cumulative PnL vs SP500 Cumulative PnL

In [63]:
import matplotlib.pyplot as plt
import seaborn as sns


# Calculate the cumulative returns
portfolio_cumulative = pd.DataFrame(index = sp500_PnL.index, columns=[f'Naive Portfolio Performance in {year}'], data=pnl_cumul)

# Set Seaborn style
sns.set(style="whitegrid")

# Plot the relative performance using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(data=pd.DataFrame(pnl_cumul))
sns.lineplot(data=sp500_PnL.cumprod(), label=f'S&P 500 Index in {year}')
plt.xlabel('Days')
plt.ylabel('Cumulated Profits and Losses')
plt.title(f'Portfolio Relative Performance vs S&P 500 in {year} - {clustering_method}')
plt.legend()
plt.show()

TypeError: tz must be string or tzinfo subclass, not <matplotlib.category.UnitData object at 0x1a9b14950>.

Error in callback <function flush_figures at 0x179452980> (for post_execute):


KeyboardInterrupt: 